# 요약, Summarization
- [강좌링크](https://wikidocs.net/166831)

가장 어려운 NLP 작업 중 하나

[aihub 도서자료 요약 데이터셋](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=93) 사용
- datas/KoT5
    - 도서자료 요약
        - Training
            - [원천]도서요약_train.zip
                - 기술과학
                    - *.json
                - 기타
                    - *.json
                - 사회과학
                    - *.json
                - 예술
                    - *.json
        - Validation
            - [원천]도서요약_valid.zip
                - 기술과학
                    - *.json
                - 기타
                    - *.json
                - 사회과학
                    - *.json
                - 예술
                    - *.json



# Preprocess

모든 json 파일 읽어서 한 csv 파일로 저장

###### columns
passage > review_body
summary > review_title

In [1]:
from tqdm.auto import tqdm
import json, os, re
import pandas as pd

base_dir = "../../datas/KoT5/도서자료 요약"
training_dir = f"{base_dir}/Training/[원천]도서요약_train"
valid_dir = f"{base_dir}/Validation/[원천]도서요약_valid"
sub_dirs = ["기술과학", "기타", "사회과학", "예술"]

def make_dataset(root_dir: str, output_file_name: str) -> None:
    column_names = ["review_title", "review_body"]
    row_data = []
    
    for sub_dir in sub_dirs:
        json_dir = f"{root_dir}/{sub_dir}"
        print(f">>> {json_dir} ...")
        
        json_files = [json_file for json_file in os.listdir(json_dir) if json_file.endswith(".json")]
        progress_bar = tqdm(range(len(json_files)))
        
        for json_file in json_files:
            with open(f"{json_dir}/{json_file}", "r") as jp:
                review = json.load(jp)
            summary = review["summary"].replace("\t", "").strip()
            passage = review["passage"].replace("\t", "").strip()
            row_data.append((summary, passage))
            progress_bar.update(1)
    
    dataset = pd.DataFrame.from_records(row_data, columns = column_names)
    dataset.to_json(output_file_name, orient = "records")

make_dataset(training_dir, "../../datas/KoT5/train_dataset.json")
make_dataset(valid_dir, "../../datas/KoT5/valid_dataset.json")

>>> ../../datas/KoT5/도서자료 요약/Training/[원천]도서요약_train/기술과학 ...


  0%|          | 0/23918 [00:00<?, ?it/s]

>>> ../../datas/KoT5/도서자료 요약/Training/[원천]도서요약_train/기타 ...


  0%|          | 0/6753 [00:00<?, ?it/s]

>>> ../../datas/KoT5/도서자료 요약/Training/[원천]도서요약_train/사회과학 ...


...

  0%|          | 0/844 [00:00<?, ?it/s]

>>> ../../datas/KoT5/도서자료 요약/Validation/[원천]도서요약_valid/사회과학 ...


  0%|          | 0/14430 [00:00<?, ?it/s]

>>> ../../datas/KoT5/도서자료 요약/Validation/[원천]도서요약_valid/예술 ...


  0%|          | 0/1736 [00:00<?, ?it/s]

In [2]:
from datasets import load_dataset
data_files = {
        "train": "../../datas/KoT5/train_dataset.json",
        "validation": "../../datas/KoT5/valid_dataset.json"
}
datasets = load_dataset("json", data_files = data_files)
datasets

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['review_body', 'review_title'],
        num_rows: 160002
    })
    validation: Dataset({
        features: ['review_body', 'review_title'],
        num_rows: 19999
    })
})

In [6]:
datasets["train"][0]

{'review_body': '지금의 국가가 직면하는 위기는 전통사회의 그것과 위기의 규모뿐만아니라 종류와 성질 면에서 사뭇 다르다고 할 수 있다. 즉 현대 사회에서는 매우 다양하고 광범위하게 위기가 발생할 뿐만 아니라 지구온난화에 따른 각종 자연재해로부터 인위적 재난인 사회적 이슈로 인한 위기에 이르기까지 복잡한 양상을 보이고 있다. 따라서 위기의 영향은 한 산업뿐만 아니라 국가의 재난으로 이어지는 경우가 많다. 따라서 국가는 이러한 위기상황이 발생하였을 경우에 그 징후를 빨리 파악하고 대응하지 않으면 안 되는데, 국민의 생명과 재산을 보호하는 것이 바로 국가의 가장 기본적인 의무이기 때문이다.',
 'review_title': '현대 사회에서 발생하는 다양하고 복잡한 위기는 과거와 다르게 국가의 재난으로 이어질 수 있으므로 국가의 신속한 대책 능력이 필요하다.'}